In [1]:
import polars as pl
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from time import perf_counter


In [2]:
orig_x, orig_y = make_classification(n_samples = 100_000, n_features = 100, n_informative = 60, n_redundant = 40)
# This is a Polars dataframe. This is dsds package's favored dataframe. dsds relies on Polars heavily.
# You must turn other dataframe formats into Polars for dsds to work.
df = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y)) 
# Turn it into Pandas.
df_pd = df.to_pandas()
target = "target"
features = df.columns
features.remove(target)

In [3]:
df.head()

target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,…,column_63,column_64,column_65,column_66,column_67,column_68,column_69,column_70,column_71,column_72,column_73,column_74,column_75,column_76,column_77,column_78,column_79,column_80,column_81,column_82,column_83,column_84,column_85,column_86,column_87,column_88,column_89,column_90,column_91,column_92,column_93,column_94,column_95,column_96,column_97,column_98,column_99
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,5.986819,0.546596,1.648701,3.997167,7.458675,-10.106747,-1.624208,1.137966,-1.158476,2.997509,4.086229,-1.26675,-1.777032,-28.261351,30.073757,25.743075,-6.093473,0.064229,-0.318723,13.616268,-6.754464,-1.317529,3.310598,0.592153,0.423041,-2.356806,-33.492711,-2.162182,-1.672547,-0.452053,7.331551,19.98831,-3.633311,5.3967,15.262042,6.747133,…,-0.355568,-20.007311,0.619315,-7.78998,2.763447,3.997654,-0.956993,48.206937,3.92971,0.351252,-2.083404,-1.703559,4.076559,-2.069447,0.184148,0.450871,-5.936437,5.175711,-4.965513,-2.766612,-5.352892,15.179981,31.906209,0.483883,14.708496,-21.290712,-1.581163,-1.675474,4.842297,-12.407528,-2.956569,2.583529,-13.347925,17.690473,10.234511,6.890357,18.68473
1,4.147852,-10.166797,2.42494,-8.280883,0.694632,2.354545,-3.867516,4.401109,-8.34216,1.981496,-8.092609,2.311992,-1.925248,-53.891539,-12.550992,-15.345019,16.917992,7.096581,1.1973,-20.604139,-6.664765,-8.888499,-2.397954,4.17647,21.591129,-5.005256,-16.221116,0.459742,-0.690709,-0.269417,-3.044781,-25.934654,-0.610498,-7.233787,2.394237,1.998173,…,2.411827,10.014729,-5.52803,-32.598132,1.486695,33.294182,-4.239926,2.168909,-5.448569,7.679037,0.707344,-58.678729,12.111013,-13.529477,3.332116,0.214316,-0.885962,-0.134195,2.699072,3.252907,-29.258296,1.955882,54.63539,9.576148,3.437032,-0.501529,-18.291438,3.040755,-3.619622,5.837794,0.914736,-3.164613,8.694552,40.241724,-50.204775,0.888179,18.313636
1,-1.065503,-3.389961,-3.420557,0.969395,-0.685428,1.423848,-0.437088,-1.246797,10.319519,-3.428728,2.236986,-0.177996,-34.727854,6.973909,19.521618,0.671872,-30.380552,2.287781,7.248455,34.126478,-10.555192,1.861879,-0.576845,-5.463621,0.438259,-1.040704,6.654079,2.672152,2.830621,2.811892,-13.247055,-28.399984,-7.399742,0.051395,10.644678,-1.586821,…,-1.996005,13.915336,1.952686,6.921012,-0.136747,11.963771,1.013418,-9.571991,0.928034,-5.626219,7.528923,46.134983,15.170741,3.778057,8.584322,-7.137716,8.735825,2.60461,3.386294,-1.134204,-18.723313,2.136473,-20.611045,-0.88672,10.00448,51.431367,-6.196474,-1.776183,2.446435,-27.402815,3.859099,0.076166,2.663502,-15.066364,14.214258,-4.997656,-10.976358
1,-6.161803,-0.062358,0.309762,-1.479081,3.194638,0.56878,1.58343,7.261861,-0.902904,2.114949,-7.247418,2.592263,33.789852,-1.67435,-7.637536,9.335591,-21.62965,0.534352,4.499181,-11.933576,-3.985087,-3.348656,-7.057779,4.619226,-41.132171,1.753467,-1.937991,-9.15416,3.534515,0.43227,1.178357,-6.930231,5.532963,24.726412,-28.003835,-0.174675,…,2.075758,6.714418,6.689934,7.808172,1.958908,6.354117,-1.641288,26.176752,2.580099,0.569722,-6.041869,10.712468,-7.501954,0.153475,6.695536,-4.308605,-9.99083,-1.252156,-3.926162,10.932434,-3.135962,-32.845303,15.726288,-2.7377,-28.695589,28.466205,-8.143614,-4.502438,0.367855,-5.163827,-3.900286,3.373313,-11.310146,2.678528,-7.299003,-5.557134,-21.532237
0,-9.346525,-1.482052,-3.0823,4.833217,4.801851,3.362576,-0.04595,-6.515114,-0.946761,-8.299702,8.758147,-7.065284,-19.829434,11.72991,-15.41

## Comparisons

This notebook compares results and performance between the dsds package, sklearn and some other packages for feature selection and some other transformations common in the data science pipeline.

### Methods Compared:
1. Scaling and Imputation
2. Fscore
3. Mutual Information Score
4. MRMR feature selection strategies
5. Power Transform

You may restart the kernel after each section. But remember to rerun the cells above. If you are concerned about memory usage when running this notebook, go to the end and run the gc cell.

# Scaling and Imputation

In [4]:
import dsds.transform as t

In [5]:
features = df.columns
features.remove("target")

In [6]:
df.head()

target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,…,column_63,column_64,column_65,column_66,column_67,column_68,column_69,column_70,column_71,column_72,column_73,column_74,column_75,column_76,column_77,column_78,column_79,column_80,column_81,column_82,column_83,column_84,column_85,column_86,column_87,column_88,column_89,column_90,column_91,column_92,column_93,column_94,column_95,column_96,column_97,column_98,column_99
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,5.986819,0.546596,1.648701,3.997167,7.458675,-10.106747,-1.624208,1.137966,-1.158476,2.997509,4.086229,-1.26675,-1.777032,-28.261351,30.073757,25.743075,-6.093473,0.064229,-0.318723,13.616268,-6.754464,-1.317529,3.310598,0.592153,0.423041,-2.356806,-33.492711,-2.162182,-1.672547,-0.452053,7.331551,19.98831,-3.633311,5.3967,15.262042,6.747133,…,-0.355568,-20.007311,0.619315,-7.78998,2.763447,3.997654,-0.956993,48.206937,3.92971,0.351252,-2.083404,-1.703559,4.076559,-2.069447,0.184148,0.450871,-5.936437,5.175711,-4.965513,-2.766612,-5.352892,15.179981,31.906209,0.483883,14.708496,-21.290712,-1.581163,-1.675474,4.842297,-12.407528,-2.956569,2.583529,-13.347925,17.690473,10.234511,6.890357,18.68473
1,4.147852,-10.166797,2.42494,-8.280883,0.694632,2.354545,-3.867516,4.401109,-8.34216,1.981496,-8.092609,2.311992,-1.925248,-53.891539,-12.550992,-15.345019,16.917992,7.096581,1.1973,-20.604139,-6.664765,-8.888499,-2.397954,4.17647,21.591129,-5.005256,-16.221116,0.459742,-0.690709,-0.269417,-3.044781,-25.934654,-0.610498,-7.233787,2.394237,1.998173,…,2.411827,10.014729,-5.52803,-32.598132,1.486695,33.294182,-4.239926,2.168909,-5.448569,7.679037,0.707344,-58.678729,12.111013,-13.529477,3.332116,0.214316,-0.885962,-0.134195,2.699072,3.252907,-29.258296,1.955882,54.63539,9.576148,3.437032,-0.501529,-18.291438,3.040755,-3.619622,5.837794,0.914736,-3.164613,8.694552,40.241724,-50.204775,0.888179,18.313636
1,-1.065503,-3.389961,-3.420557,0.969395,-0.685428,1.423848,-0.437088,-1.246797,10.319519,-3.428728,2.236986,-0.177996,-34.727854,6.973909,19.521618,0.671872,-30.380552,2.287781,7.248455,34.126478,-10.555192,1.861879,-0.576845,-5.463621,0.438259,-1.040704,6.654079,2.672152,2.830621,2.811892,-13.247055,-28.399984,-7.399742,0.051395,10.644678,-1.586821,…,-1.996005,13.915336,1.952686,6.921012,-0.136747,11.963771,1.013418,-9.571991,0.928034,-5.626219,7.528923,46.134983,15.170741,3.778057,8.584322,-7.137716,8.735825,2.60461,3.386294,-1.134204,-18.723313,2.136473,-20.611045,-0.88672,10.00448,51.431367,-6.196474,-1.776183,2.446435,-27.402815,3.859099,0.076166,2.663502,-15.066364,14.214258,-4.997656,-10.976358
1,-6.161803,-0.062358,0.309762,-1.479081,3.194638,0.56878,1.58343,7.261861,-0.902904,2.114949,-7.247418,2.592263,33.789852,-1.67435,-7.637536,9.335591,-21.62965,0.534352,4.499181,-11.933576,-3.985087,-3.348656,-7.057779,4.619226,-41.132171,1.753467,-1.937991,-9.15416,3.534515,0.43227,1.178357,-6.930231,5.532963,24.726412,-28.003835,-0.174675,…,2.075758,6.714418,6.689934,7.808172,1.958908,6.354117,-1.641288,26.176752,2.580099,0.569722,-6.041869,10.712468,-7.501954,0.153475,6.695536,-4.308605,-9.99083,-1.252156,-3.926162,10.932434,-3.135962,-32.845303,15.726288,-2.7377,-28.695589,28.466205,-8.143614,-4.502438,0.367855,-5.163827,-3.900286,3.373313,-11.310146,2.678528,-7.299003,-5.557134,-21.532237
0,-9.346525,-1.482052,-3.0823,4.833217,4.801851,3.362576,-0.04595,-6.515114,-0.946761,-8.299702,8.758147,-7.065284,-19.829434,11.72991,-15.41

In [7]:
df_pd.head()

,target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,...,column_90,column_91,column_92,column_93,column_94,column_95,column_96,column_97,column_98,column_99
0,1,5.986819,0.546596,1.648701,3.997167,7.458675,-10.106747,-1.624208,1.137966,-1.158476,...,-1.675474,4.842297,-12.407528,-2.956569,2.583529,-13.347925,17.690473,10.234511,6.890357,18.684730
1,1,4.147852,-10.166797,2.424940,-8.280883,0.694632,2.354545,-3.867516,4.401109,-8.342160,...,3.040755,-3.619622,5.837794,0.914736,-3.164613,8.694552,40.241724,-50.204775,0.888179,18.313636
2,1,-1.065503,-3.389961,-3.420557,0.969395,-0.685428,1.423848,-0.437088,-1.246797,10.319519,...,-1.776183,2.446435,-27.402815,3.859099,0.076166,2.663502,-15.066364,14.214258,-4.997656,-10.976358
3,1,-6.161803,-0.062358,0.309762,-1.479081,3.194638,0.568780,1.583430,7.261861,-0.902904,...,-4.502438,0.367855,-5.163827,-3.900286,3.373313,-11.310146,2.678528,-7.299003,-5.557134,-21.532237
4,0,-9.346525,-1.482052,-3.082300,4.833217,4.801851,3.362576,-0.045950,-6.515114,-0.946761,...,-6.494197,-3.668589,6.325084,1.606812,3.314172,17.470099,-26.354451,-11.840262,-1.798413,-6.893892


In [8]:
scaled = t.scale(df, cols=features, strategy="normal")
scaled.head()

target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,…,column_63,column_64,column_65,column_66,column_67,column_68,column_69,column_70,column_71,column_72,column_73,column_74,column_75,column_76,column_77,column_78,column_79,column_80,column_81,column_82,column_83,column_84,column_85,column_86,column_87,column_88,column_89,column_90,column_91,column_92,column_93,column_94,column_95,column_96,column_97,column_98,column_99
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,1.294905,0.333453,0.352402,1.108324,1.479123,-2.306337,-0.238395,0.359756,-0.361,0.574575,1.00125,-0.158437,-0.120713,-1.357705,1.461795,1.413173,-0.304733,0.124442,-0.287602,0.615356,-1.285307,-0.175165,0.831514,0.134627,0.232212,-0.64412,-1.857767,-0.260619,-0.260622,-0.215072,0.292783,1.009219,-0.591925,0.067739,0.672425,1.308126,…,0.036962,-1.0899,0.140539,-0.410279,0.606058,0.062623,-0.103909,2.61926,0.658685,0.311498,-0.344515,0.035201,0.438634,-0.331842,0.043715,0.203418,-1.385078,1.372731,-1.071483,-0.538954,-0.070333,0.617229,1.579176,-0.000021,0.767606,-1.141811,0.094667,-0.252523,0.879117,-0.477686,-0.615769,0.542548,-0.605181,0.756591,0.69854,1.585823,1.030014
1,0.898777,-1.974316,0.521609,-1.615605,-0.066947,0.541733,-0.722267,1.079211,-1.927822,0.342838,-1.675549,0.611185,-0.127879,-2.753463,-0.660457,-0.741392,0.874554,1.597954,0.044165,-1.155583,-1.265199,-1.772964,-0.421137,0.928928,1.217453,-1.244188,-0.989669,0.328591,-0.04569,-0.173376,-0.249166,-1.283373,0.087737,-0.494102,0.059516,0.227629,…,0.671639,0.534542,-1.2431,-1.853319,0.328788,1.458983,-0.851963,0.020185,-1.460235,1.992302,0.254837,-2.688321,0.909258,-2.746669,0.752187,0.152638,-0.295849,0.188946,0.579498,0.880194,-1.269484,-0.088848,2.634337,2.015798,0.184176,-0.039079,-0.803603,0.776048,-1.051954,0.388622,0.189535,-0.67145,0.529248,1.903783,-2.402292,0.298147,1.011409
1,-0.22422,-0.514519,-0.752611,0.436602,-0.38239,0.329019,0.017661,-0.166034,2.142446,-0.891152,0.594803,0.075704,-1.713767,0.561123,0.936412,0.098497,-1.549393,0.59035,1.368402,1.67678,-2.137339,0.495827,-0.021523,-1.20736,0.23292,-0.345926,0.160075,0.825772,0.725157,0.530094,-0.782024,-1.406449,-1.438785,-0.170036,0.452494,-0.588039,…,-0.339258,0.745598,0.440653,0.445429,-0.023772,0.442312,0.345071,-0.642647,-0.019511,-1.059579,1.719866,2.321975,1.088484,0.900328,1.934233,-1.425577,1.779267,0.799532,0.727529,-0.154101,-0.741024,-0.079206,-0.858842,-0.303893,0.524118,2.715627,-0.153432,-0.274487,0.332364,-1.189679,0.802017,0.012997,0.218857,-0.909764,0.90272,-0.964569,-0.457124
1,-1.322002,0.202279,0.060536,-0.106601,0.504484,0.13359,0.453478,1.709947,-0.305258,0.373277,-1.489784,0.671458,1.598818,0.09016,-0.41582,0.552802,-1.100928,0.222948,0.76675,-0.706873,-0.66448,-0.603819,-1.44366,1.027044,-1.701921,0.287158,-0.271776,-1.831884,0.879245,-0.013179,-0.028594,-0.334624,1.469059,0.927582,-1.38838,-0.266744,…,0.594564,0.355968,1.506908,0.497034,0.431338,0.174939,-0.259834,1.375547,0.353755,0.361609,-1.19465,0.628711,-0.239586,0.136566,1.509149,-0.818269,-2.259483,-0.060291,-0.847602,2.6907,0.040873,-1.946996,0.828053,-0.714268,-1.479062,1.497473,-0.258101,-0.869062,-0.141983,-0.133747,-0.81208,0.709349,-0.500305,-0.007074,-0.201015,-1.084597,-0.986371
0,-2.008016,-0.103537,-0.678877,1.293805,0.871848,0.772122,0.102028,-1.327589,-0.314823,-2.002148,2.028095,-1.405433,-0.993482,0.820124,-0.803125,1.004748,-0.837777,-0.191669,0.804184,-0.113826,0.924836,0.330462,-0.338266,

In [9]:
from sklearn.preprocessing import StandardScaler

In [10]:
# The difference in result is caused by using ddof = 1 for sample variance in dsds
# and using ddof = 0 in sklearn.

# Long and convoluted code just to do some scaling...
std = StandardScaler()
scaled2 = std.fit_transform(df_pd[features], df_pd[target])
# scaled2[:5, :] # scaled2 is a numpy matrix
scaled2 = pd.DataFrame(scaled2, columns=features)
scaled2[target] = df_pd[target]
scaled2.head()

,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,...,column_91,column_92,column_93,column_94,column_95,column_96,column_97,column_98,column_99,target
0,1.294911,0.333455,0.352404,1.108330,1.479130,-2.306348,-0.238396,0.359758,-0.361002,0.574578,...,0.879121,-0.477688,-0.615772,0.542550,-0.605184,0.756594,0.698543,1.585831,1.030020,1
1,0.898781,-1.974326,0.521612,-1.615613,-0.066947,0.541735,-0.722270,1.079216,-1.927831,0.342840,...,-1.051960,0.388624,0.189536,-0.671453,0.529251,1.903792,-2.402304,0.298149,1.011414,1
2,-0.224221,-0.514522,-0.752615,0.436604,-0.382392,0.329021,0.017661,-0.166035,2.142457,-0.891157,...,0.332366,-1.189684,0.802021,0.012997,0.218858,-0.909769,0.902725,-0.964574,-0.457126,1
3,-1.322008,0.202280,0.060536,-0.106601,0.504487,0.133591,0.453480,1.709956,-0.305260,0.373279,...,-0.141984,-0.133748,-0.812084,0.709352,-0.500308,-0.007074,-0.201016,-1.084602,-0.986376,1
4,-2.008026,-0.103538,-0.678880,1.293811,0.871852,0.772125,0.102028,-1.327595,-0.314825,-2.002158,...,-1.063134,0.411761,0.333501,0.696862,0.980892,-1.484001,-0.434006,-0.278222,-0.252439,0


In [11]:
%%timeit
scaled = t.scale(df, cols=features, strategy="normal")

23.4 ms ± 411 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%%timeit
std = StandardScaler()
scaled2 = std.fit_transform(df_pd[features], df_pd[target])
scaled2 = pd.DataFrame(scaled2, columns=features)
scaled2[target] = df_pd[target]


130 ms ± 4.89 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
t.impute(df, cols=features, strategy="median").head() # 8.29 ms

target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,…,column_63,column_64,column_65,column_66,column_67,column_68,column_69,column_70,column_71,column_72,column_73,column_74,column_75,column_76,column_77,column_78,column_79,column_80,column_81,column_82,column_83,column_84,column_85,column_86,column_87,column_88,column_89,column_90,column_91,column_92,column_93,column_94,column_95,column_96,column_97,column_98,column_99
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,5.986819,0.546596,1.648701,3.997167,7.458675,-10.106747,-1.624208,1.137966,-1.158476,2.997509,4.086229,-1.26675,-1.777032,-28.261351,30.073757,25.743075,-6.093473,0.064229,-0.318723,13.616268,-6.754464,-1.317529,3.310598,0.592153,0.423041,-2.356806,-33.492711,-2.162182,-1.672547,-0.452053,7.331551,19.98831,-3.633311,5.3967,15.262042,6.747133,…,-0.355568,-20.007311,0.619315,-7.78998,2.763447,3.997654,-0.956993,48.206937,3.92971,0.351252,-2.083404,-1.703559,4.076559,-2.069447,0.184148,0.450871,-5.936437,5.175711,-4.965513,-2.766612,-5.352892,15.179981,31.906209,0.483883,14.708496,-21.290712,-1.581163,-1.675474,4.842297,-12.407528,-2.956569,2.583529,-13.347925,17.690473,10.234511,6.890357,18.68473
1,4.147852,-10.166797,2.42494,-8.280883,0.694632,2.354545,-3.867516,4.401109,-8.34216,1.981496,-8.092609,2.311992,-1.925248,-53.891539,-12.550992,-15.345019,16.917992,7.096581,1.1973,-20.604139,-6.664765,-8.888499,-2.397954,4.17647,21.591129,-5.005256,-16.221116,0.459742,-0.690709,-0.269417,-3.044781,-25.934654,-0.610498,-7.233787,2.394237,1.998173,…,2.411827,10.014729,-5.52803,-32.598132,1.486695,33.294182,-4.239926,2.168909,-5.448569,7.679037,0.707344,-58.678729,12.111013,-13.529477,3.332116,0.214316,-0.885962,-0.134195,2.699072,3.252907,-29.258296,1.955882,54.63539,9.576148,3.437032,-0.501529,-18.291438,3.040755,-3.619622,5.837794,0.914736,-3.164613,8.694552,40.241724,-50.204775,0.888179,18.313636
1,-1.065503,-3.389961,-3.420557,0.969395,-0.685428,1.423848,-0.437088,-1.246797,10.319519,-3.428728,2.236986,-0.177996,-34.727854,6.973909,19.521618,0.671872,-30.380552,2.287781,7.248455,34.126478,-10.555192,1.861879,-0.576845,-5.463621,0.438259,-1.040704,6.654079,2.672152,2.830621,2.811892,-13.247055,-28.399984,-7.399742,0.051395,10.644678,-1.586821,…,-1.996005,13.915336,1.952686,6.921012,-0.136747,11.963771,1.013418,-9.571991,0.928034,-5.626219,7.528923,46.134983,15.170741,3.778057,8.584322,-7.137716,8.735825,2.60461,3.386294,-1.134204,-18.723313,2.136473,-20.611045,-0.88672,10.00448,51.431367,-6.196474,-1.776183,2.446435,-27.402815,3.859099,0.076166,2.663502,-15.066364,14.214258,-4.997656,-10.976358
1,-6.161803,-0.062358,0.309762,-1.479081,3.194638,0.56878,1.58343,7.261861,-0.902904,2.114949,-7.247418,2.592263,33.789852,-1.67435,-7.637536,9.335591,-21.62965,0.534352,4.499181,-11.933576,-3.985087,-3.348656,-7.057779,4.619226,-41.132171,1.753467,-1.937991,-9.15416,3.534515,0.43227,1.178357,-6.930231,5.532963,24.726412,-28.003835,-0.174675,…,2.075758,6.714418,6.689934,7.808172,1.958908,6.354117,-1.641288,26.176752,2.580099,0.569722,-6.041869,10.712468,-7.501954,0.153475,6.695536,-4.308605,-9.99083,-1.252156,-3.926162,10.932434,-3.135962,-32.845303,15.726288,-2.7377,-28.695589,28.466205,-8.143614,-4.502438,0.367855,-5.163827,-3.900286,3.373313,-11.310146,2.678528,-7.299003,-5.557134,-21.532237
0,-9.346525,-1.482052,-3.0823,4.833217,4.801851,3.362576,-0.04595,-6.515114,-0.946761,-8.299702,8.758147,-7.065284,-19.829434,11.72991,-15.41

In [14]:
from sklearn.impute import SimpleImputer # 1.25s

imputer = SimpleImputer(strategy = "median")
imputed = pd.DataFrame(imputer.fit_transform(df_pd, df_pd[target]), columns=df.columns)
imputed.head()

,target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,...,column_90,column_91,column_92,column_93,column_94,column_95,column_96,column_97,column_98,column_99
0,1.0,5.986819,0.546596,1.648701,3.997167,7.458675,-10.106747,-1.624208,1.137966,-1.158476,...,-1.675474,4.842297,-12.407528,-2.956569,2.583529,-13.347925,17.690473,10.234511,6.890357,18.684730
1,1.0,4.147852,-10.166797,2.424940,-8.280883,0.694632,2.354545,-3.867516,4.401109,-8.342160,...,3.040755,-3.619622,5.837794,0.914736,-3.164613,8.694552,40.241724,-50.204775,0.888179,18.313636
2,1.0,-1.065503,-3.389961,-3.420557,0.969395,-0.685428,1.423848,-0.437088,-1.246797,10.319519,...,-1.776183,2.446435,-27.402815,3.859099,0.076166,2.663502,-15.066364,14.214258,-4.997656,-10.976358
3,1.0,-6.161803,-0.062358,0.309762,-1.479081,3.194638,0.568780,1.583430,7.261861,-0.902904,...,-4.502438,0.367855,-5.163827,-3.900286,3.373313,-11.310146,2.678528,-7.299003,-5.557134,-21.532237
4,0.0,-9.346525,-1.482052,-3.082300,4.833217,4.801851,3.362576,-0.045950,-6.515114,-0.946761,...,-6.494197,-3.668589,6.325084,1.606812,3.314172,17.470099,-26.354451,-11.840262,-1.798413,-6.893892


In [15]:
%%timeit
t.impute(df, cols=features, strategy="median")

8.29 ms ± 56.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
%%timeit
imputer = SimpleImputer(strategy = "median")
imputed = pd.DataFrame(imputer.fit_transform(df_pd, df_pd[target]), columns=df.columns)

1.25 s ± 9.45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Fscore

In [17]:
import dsds.fs as fs # fs = feature_selection
from sklearn.feature_selection import mutual_info_classif, f_classif, f_regression

In [18]:
start = perf_counter()
res = fs.f_classif(df, target=target)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing Fscore.")
res.head(10)

Spent 0.02s in computing Fscore.


feature,f_value,p_value
str,f64,f64
"""column_0""",1.147982,0.283975
"""column_1""",8.6081e-7,0.99926
"""column_2""",0.160483,0.688713
"""column_3""",0.381542,0.536781
"""column_4""",0.00059,0.980622
"""column_5""",0.020556,0.885995
"""column_6""",1205.834997,1.2189e-262
"""column_7""",1199.187486,3.2619e-261
"""column_8""",1211.524308,7.3161e-264


In [19]:
start = perf_counter()
f, pv = f_classif(df_pd[features], df_pd[target])
res = pd.DataFrame({"feature":features, "f_value":f, "p_value":pv})
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing Fscore.")
res.head(10)

Spent 0.12s in computing Fscore.


,feature,f_value,p_value
0,column_0,1.147982e+00,2.839746e-01
1,column_1,8.608100e-07,9.992597e-01
2,column_2,1.604831e-01,6.887130e-01
3,column_3,3.815420e-01,5.367805e-01
4,column_4,5.899596e-04,9.806221e-01
5,column_5,2.055620e-02,8.859948e-01
6,column_6,1.205835e+03,1.218853e-262
7,column_7,1.199187e+03,3.261909e-261
8,column_8,1.211524e+03,7.316090e-264
9,column_9,1.262539e+03,8.186518e-275


In [20]:
start = perf_counter()
f, pv = f_regression(df_pd[features], df_pd[target])
res = pd.DataFrame({"feature":features, "f_value":f, "p_value":pv})
end = perf_counter()
print(f"Spent {end - start: .2f}s in computing Fscore.")
res.head(10)

Spent  0.05s in computing Fscore.


,feature,f_value,p_value
0,column_0,1.147982e+00,2.839746e-01
1,column_1,8.608105e-07,9.992597e-01
2,column_2,1.604831e-01,6.887130e-01
3,column_3,3.815420e-01,5.367805e-01
4,column_4,5.899596e-04,9.806221e-01
5,column_5,2.055620e-02,8.859948e-01
6,column_6,1.205835e+03,1.218853e-262
7,column_7,1.199187e+03,3.261909e-261
8,column_8,1.211524e+03,7.316090e-264
9,column_9,1.262539e+03,8.186518e-275


# Mutual Information Score

In [21]:
# Took 7.4s
fs.mutual_info(df, target=target, conti_cols=features).sort(by="estimated_mi", descending=True).limit(10)

Mutual Info: 100%|██████████| 100/100 [00:07<00:00, 13.57it/s]


feature,estimated_mi
str,f64
"""column_47""",0.027364
"""column_93""",0.022182
"""column_40""",0.016389
"""column_99""",0.015347
"""column_33""",0.014182
"""column_29""",0.013522
"""column_50""",0.012938
"""column_38""",0.012355
"""column_37""",0.011551


In [22]:
def estimate_mi_sklearn(df:pd.DataFrame, cols:list[str], target:str, k=3, random_state:int=42):
    mi_estimates = mutual_info_classif(df[cols], df[target]
                        , n_neighbors=k, random_state=random_state, discrete_features=False)

    return pl.from_records([cols, mi_estimates], schema=["feature", "estimated_mi"]).sort("estimated_mi", descending=True)

In [23]:
# Took 1m38s seconds by using sklearn. The reason is that sklearn's implementation did not turn on multithreading for KDtrees.
# Sklearn also did not provide an option to turn it on, despite the fact that sklearn's KDtrees
# does have this functionality.
estimate_mi_sklearn(df_pd, cols=features, target=target).limit(10)

feature,estimated_mi
str,f64
"""column_47""",0.027364
"""column_93""",0.022182
"""column_40""",0.016391
"""column_99""",0.015345
"""column_33""",0.014182
"""column_29""",0.013522
"""column_50""",0.01294
"""column_38""",0.012355
"""column_37""",0.01155


# MRMR Feature selection Strategy

In [24]:
from mrmr import mrmr_classif # This is currently the most starred MRMR Python package on github

In [25]:
# Need to wrap it so that we get apples to apples comparison
def mrmr_package(df:pd.DataFrame, target:str, k:int) -> list[str]:
    features = list(df.columns)
    features.remove(target)
    X = df[features]
    y = df[target]
    start = perf_counter()
    output = mrmr_classif(X, y, K = k)
    end = perf_counter()
    print(f"Spent {end - start:.2f}s to compute mrmr.")
    return output

In [26]:
mrmr_package(df_pd, "target", 50)

100%|██████████| 50/50 [00:10<00:00,  4.60it/s]

Spent 14.31s to compute mrmr.


['column_47',
 'column_63',
 'column_78',
 'column_54',
 'column_93',
 'column_79',
 'column_48',
 'column_69',
 'column_39',
 'column_10',
 'column_98',
 'column_17',
 'column_29',
 'column_86',
 'column_6',
 'column_33',
 'column_9',
 'column_82',
 'column_55',
 'column_21',
 'column_13',
 'column_37',
 'column_25',
 'column_76',
 'column_40',
 'column_22',
 'column_11',
 'column_28',
 'column_8',
 'column_73',
 'column_56',
 'column_52',
 'column_36',
 'column_90',
 'column_7',
 'column_70',
 'column_50',
 'column_38',
 'column_99',
 'column_41',
 'column_83',
 'column_45',
 'column_96',
 'column_34',
 'column_97',
 'column_26',
 'column_42',
 'column_87',
 'column_68',
 'column_84']

In [27]:
start = perf_counter()
res = fs.mrmr(df, target="target", k = 50, low_memory=False)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res

Running fscore to determine feature relevance...
Found 100 total features to select from. Proceeding to select top 50 features.


MRMR, fscore: 100%|██████████| 50/50 [00:00<00:00, 195.27it/s]

Output is sorted in order of selection (relevance).
Spent 0.30s in computing.


['column_47',
 'column_63',
 'column_78',
 'column_54',
 'column_93',
 'column_79',
 'column_48',
 'column_69',
 'column_39',
 'column_10',
 'column_98',
 'column_17',
 'column_29',
 'column_86',
 'column_6',
 'column_33',
 'column_9',
 'column_82',
 'column_55',
 'column_21',
 'column_13',
 'column_37',
 'column_25',
 'column_76',
 'column_40',
 'column_22',
 'column_11',
 'column_28',
 'column_8',
 'column_73',
 'column_56',
 'column_52',
 'column_36',
 'column_90',
 'column_7',
 'column_70',
 'column_50',
 'column_38',
 'column_99',
 'column_41',
 'column_83',
 'column_45',
 'column_96',
 'column_34',
 'column_97',
 'column_26',
 'column_42',
 'column_87',
 'column_68',
 'column_84']

In [28]:
start = perf_counter()
res = fs.mrmr(df, target="target", k = 50, low_memory=True)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res

Running fscore to determine feature relevance...
Found 100 total features to select from. Proceeding to select top 50 features.


MRMR, fscore: 100%|██████████| 50/50 [00:01<00:00, 46.76it/s]

Output is sorted in order of selection (relevance).
Spent 1.09s in computing.


['column_47',
 'column_63',
 'column_78',
 'column_54',
 'column_93',
 'column_79',
 'column_48',
 'column_69',
 'column_39',
 'column_10',
 'column_98',
 'column_17',
 'column_29',
 'column_86',
 'column_6',
 'column_33',
 'column_9',
 'column_82',
 'column_55',
 'column_21',
 'column_13',
 'column_37',
 'column_25',
 'column_76',
 'column_40',
 'column_22',
 'column_11',
 'column_28',
 'column_8',
 'column_73',
 'column_56',
 'column_52',
 'column_36',
 'column_90',
 'column_7',
 'column_70',
 'column_50',
 'column_38',
 'column_99',
 'column_41',
 'column_83',
 'column_45',
 'column_96',
 'column_34',
 'column_97',
 'column_26',
 'column_42',
 'column_87',
 'column_68',
 'column_84']

# Power Transform

In [29]:
# Eager transform.
start = perf_counter()
res_eager = t.power_transform(df, cols=features, strategy="yeo_johnson")
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res_eager.head() 

Inferring best paramters: 100%|██████████| 100/100 [00:01<00:00, 61.98it/s]


Spent 1.74s in computing.


target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,…,column_63,column_64,column_65,column_66,column_67,column_68,column_69,column_70,column_71,column_72,column_73,column_74,column_75,column_76,column_77,column_78,column_79,column_80,column_81,column_82,column_83,column_84,column_85,column_86,column_87,column_88,column_89,column_90,column_91,column_92,column_93,column_94,column_95,column_96,column_97,column_98,column_99
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,6.01535,0.546501,1.643836,4.002086,7.452206,-9.771592,-1.623333,1.134822,-1.164929,3.014361,4.08917,-1.261647,-1.775636,-28.118007,30.299479,25.973076,-6.294637,0.064228,-0.318748,13.56766,-6.786885,-1.320511,3.311371,0.59225,0.423113,-2.369111,-33.785555,-2.162067,-1.682431,-0.450174,7.290018,19.726521,-3.624857,5.40707,15.260811,6.728758,…,-0.356182,-20.080501,0.617387,-7.7774,2.751548,4.008117,-0.956815,50.097601,3.929083,0.351059,-2.091347,-1.705404,4.066172,-2.060925,0.184058,0.450799,-5.900022,5.167614,-4.940943,-2.765675,-5.349584,15.061502,31.952164,0.483696,14.818224,-21.352494,-1.581689,-1.686986,4.864902,-12.43688,-2.95384,2.58419,-13.517491,17.64964,10.209761,6.877143,18.462875
1,4.163948,-10.179248,2.415587,-8.265827,0.694511,2.389893,-3.86382,4.370761,-8.504408,1.989947,-8.084204,2.326432,-1.923648,-53.554437,-12.484463,-15.233791,16.062962,7.091787,1.197016,-20.691494,-6.696535,-8.95381,-2.397499,4.179303,21.63621,-5.046817,-16.329346,0.459749,-0.692747,-0.268715,-3.055319,-26.314749,-0.610116,-7.217608,2.394165,1.995562,…,2.392748,9.987499,-5.60925,-32.50317,1.482527,33.522215,-4.237712,2.18819,-5.449617,7.640678,0.706169,-59.026756,12.057204,-13.372738,3.315109,0.214299,-0.88445,-0.134207,2.708359,3.254129,-29.220057,1.950917,54.728883,9.547878,3.449203,-0.501669,-18.315018,3.009795,-3.605438,5.828239,0.915098,-3.163691,8.60486,40.117015,-50.42668,0.887741,18.097888
1,-1.063884,-3.392264,-3.437086,0.969839,-0.685547,1.438776,-0.437007,-1.250506,10.102604,-3.407911,2.238085,-0.177865,-34.605424,6.993467,19.646566,0.672563,-32.418748,2.286989,7.24303,33.953944,-10.618171,1.856469,-0.576808,-5.459312,0.438337,-1.043689,6.624811,2.672316,2.806871,2.860568,-13.346692,-28.83001,-7.37424,0.051397,10.643944,-1.588582,…,-2.010012,13.871558,1.937844,6.93156,-0.136795,12.018818,1.013615,-9.376114,0.92798,-5.650259,7.467845,45.88215,15.096535,3.801133,8.510789,-7.14567,8.80152,2.601917,3.399748,-1.134001,-18.702572,2.130709,-20.585739,-0.887293,10.067413,51.230694,-6.201338,-1.788912,2.453942,-27.491511,3.863301,0.076167,2.649376,-15.099021,14.174526,-5.005744,-11.08174
1,-6.132151,-0.06236,0.309533,-1.478143,3.192917,0.571616,1.584268,7.196294,-0.907031,2.124386,-7.240294,2.609738,33.908335,-1.672402,-7.605418,9.390299,-22.908023,0.534292,4.496577,-11.973959,-3.999367,-3.363067,-7.055254,4.622547,-41.02493,1.746052,-1.942035,-9.153041,3.500459,0.434007,1.176271,-6.987514,5.549403,24.820718,-28.006685,-0.174704,…,2.06092,6.699404,6.583106,7.820819,1.952212,6.375653,-1.640827,27.011342,2.579783,0.569243,-6.086016,10.678634,-7.528681,0.153545,6.644857,-4.312254,-9.911476,-1.252926,-3.909188,10.940284,-3.134546,-33.191383,15.743654,-2.74174,-28.421506,28.374653,-8.15096,-4.561316,0.368105,-5.171747,-3.896017,3.374336,-11.443679,2.676197,-7.314035,-5.566668,-21.805974
0,-9.291163,-1.482626,-3.096258,4.839853,4.798556,3.426212,-0.045949,-6.571324,-0.951256,-8.217974,8.767611,-6.9809,-19.77228,11.772089,-15.327262,18.0

In [30]:
from sklearn.preprocessing import power_transform

In [31]:
# Sklearn with Pandas

start = perf_counter()
transformed = power_transform(df_pd[features], method = "yeo-johnson", standardize=False)
end = perf_counter()
df_pd[features] = transformed
print(f"Spent {end - start:.2f}s in computing.")
df_pd.head()


Spent 7.54s in computing.


,target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,...,column_90,column_91,column_92,column_93,column_94,column_95,column_96,column_97,column_98,column_99
0,1,6.015350,0.546501,1.643836,4.002086,7.452206,-9.771592,-1.623333,1.134822,-1.164929,...,-1.686986,4.864902,-12.436880,-2.953840,2.584190,-13.517491,17.649640,10.209761,6.877143,18.462875
1,1,4.163948,-10.179248,2.415587,-8.265827,0.694511,2.389893,-3.863820,4.370761,-8.504408,...,3.009795,-3.605438,5.828239,0.915098,-3.163691,8.604860,40.117015,-50.426680,0.887741,18.097888
2,1,-1.063884,-3.392264,-3.437086,0.969839,-0.685547,1.438776,-0.437007,-1.250506,10.102604,...,-1.788912,2.453942,-27.491511,3.863301,0.076167,2.649376,-15.099021,14.174526,-5.005744,-11.081740
3,1,-6.132151,-0.062360,0.309533,-1.478143,3.192917,0.571616,1.584268,7.196294,-0.907031,...,-4.561316,0.368105,-5.171747,-3.896017,3.374336,-11.443679,2.676197,-7.314035,-5.566668,-21.805974
4,0,-9.291163,-1.482626,-3.096258,4.839853,4.798556,3.426212,-0.045949,-6.571324,-0.951256,...,-6.598112,-3.654096,6.314281,1.607791,3.315166,17.225647,-26.425384,-11.870934,-1.799935,-6.946730


# GC

In [32]:
import gc 
gc.collect()

94